Remarque : fait à l'arrache dans le notebook pour l'instant, déplacement dans un .py plus tard

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import math
import sys
sys.path.append("..")
from common import get_positional_encodings
import nltk

In [2]:
#entrée : positional encoding + embeddings
#ajouter layer norm

In [13]:
# Paramètres

nb_decoders = 6
nb_heads = 8
vector_size = 512
vocab_size = 1000
ffn_hidden_size = 100
seq_length = 15
attention_features_size = 5


In [11]:
class Transformer(nn.Module):
    def __init__(self, decoder):
        super(Transformer, self).__init__()
        self.decoder = decoder

    def forward(self, x):
        for x in range(nb_decoders):
            x = self.decoder(x)
        #A definir
        Traitements_posts(x)
        return x

In [12]:
class Decoder(nn.Module):
    "Decoder block applying multihead attention mechanism followed by a feedforward network"
    def __init__(self, multihead_attention, feedforward_network):
        super(Decoder, self).__init__()
        self.multihead_attention = multihead_attention
        self.feedforward_network = feedforward_network
        self.layernorm = nn.modules.normalization.LayerNorm

    def forward(self, x):
        #normalisation a la fin ou au debut ?
        mha = self.multihead_attention(self.layernorm(x))
        x += mha
        ffo = self.feedforward_network(self.layernorm(x))
        x += ffo
        return x
    

In [23]:
class FeedforwardNetwork(nn.Module):
    def __init__(self, vector_size, hidden_size):
        "Classic Feedforward Network with two layers"
        super(FeedforwardNetwork, self).__init__()
        self.hidden_size  = hidden_size
        self.fc1 = nn.Linear(vector_size, self.hidden_size)
        self.fc2 = nn.Linear(self.hidden_size, vector_size)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [24]:
class MultiHeadAttention(nn.Module):
    "MultiHead Attention Block"
    def __init__(self, nb_heads, features_size, vector_size):
        super(MultiHeadAttention, self).__init__()
        self.nb_heads = nb_heads
        self.features_size = features_size
        #A changer
        self.w_q = nn.Linear(vector_size, self.nb_heads*self.features_size)
        self.w_k = nn.Linear(vector_size, self.nb_heads*self.features_size)
        self.w_v = nn.Linear(vector_size, self.nb_heads*self.features_size)
        self.w_0 = nn.Linear(self.nb_heads*vector_size, vector_size)
        
    def mask_w(w):
        #pseudo code
        return w*(w.index() >= present)
    
    def attention(self, q, k, v):
        w = torch.matmul(q, k.transpose(1,0)) / math.sqrt() #v.shape[-1].value
        w = mask_w(w)
        w = F.softmax(w)
        a = tf.matmul(w, v)
        return a
        
    def forward(self):
        q = self.w_q(x).view(self.nb_heads, -1)
        k = self.w_k(x).view(self.nb_heads, -1)
        v = self.w_v(x).view(self.nb_heads, -1)
        a = attention(q, k, v) #redimensionner ?
        a = self.w_0(a)
        return a


In [31]:
x = "fkjernlkjzerlkgjnerz"

In [37]:
embedding = nn.Embedding(vocab_size, 4)
embedded = embedding(x)

pos_encodings = torch.tensor(get_positional_encodings(seq_length, vector_size))
encoded = embedded + pos_encodings

ValueError: too many dimensions 'str'

In [27]:
LMtransformer = Transformer(Decoder(MultiHeadAttention(nb_heads, attention_features_size, vector_size), FeedforwardNetwork(vector_size, ffn_hidden_size)))

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(LMtransformer.parameters(), lr=0.001)

In [7]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Eric\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [ ]:
with open("fr.train.top1M.txt","r",encoding="utf8") as file:
    text = file.read()

In [ ]:
tokens = nltk.word_tokenize(text)
print(tokens[0])

In [ ]:
train_txt, val_txt, test_txt = 
TEXT.build_vocab(train_txt)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def batchify(data, bsz):
    data = TEXT.numericalize([data.examples[0].text])
    # Divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)

batch_size = 20
eval_batch_size = 10
train_data = batchify(train_txt, batch_size)
val_data = batchify(val_txt, eval_batch_size)
test_data = batchify(test_txt, eval_batch_size)

In [ ]:

def train():
    LMtransformer.train()
    running_loss = 0
    ntokens = len(TEXT.vocab.stoi)
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output.view(-1, ntokens), targets)
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        log_interval = 200
        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:02.2f} | '
                  'loss {:5.2f} | ppl {:8.2f}'.format(
                    epoch, batch, len(train_data) // bptt,
                    cur_loss, math.exp(cur_loss)))
            running_loss = 0

In [ ]:

for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

In [ ]:
def evaluate(eval_model, data_source):
    eval_model.eval() # Turn on the evaluation mode
    total_loss = 0.
    ntokens = len(TEXT.vocab.stoi)
    with torch.no_grad():
        for i in range(0, data_source.size(0) - 1, bptt):
            data, targets = get_batch(data_source, i)
            output = eval_model(data)
            output_flat = output.view(-1, ntokens)
            total_loss += len(data) * criterion(output_flat, targets).item()
    return total_loss / (len(data_source) - 1)

In [ ]:
def model(hparams, X, past=None, scope='model', reuse=False):
    with tf.variable_scope(scope, reuse=reuse):
        results = {}
        batch, sequence = shape_list(X)

        wpe = tf.get_variable('wpe', [hparams.n_ctx, hparams.n_embd],
                             initializer=tf.random_normal_initializer(stddev=0.01))
        wte = tf.get_variable('wte', [hparams.n_vocab, hparams.n_embd],
                             initializer=tf.random_normal_initializer(stddev=0.02))
        past_length = 0 if past is None else tf.shape(past)[-2]
        h = tf.gather(wte, X) + tf.gather(wpe, positions_for(X, past_length))

        # Transformer
        presents = []
        pasts = tf.unstack(past, axis=1) if past is not None else [None] * hparams.n_layer
        assert len(pasts) == hparams.n_layer
        for layer, past in enumerate(pasts):
            h, present = block(h, 'h%d' % layer, past=past, hparams=hparams)
            presents.append(present)
        results['present'] = tf.stack(presents, axis=1)
        h = norm(h, 'ln_f')

        # Language model loss.  Do tokens <n predict token n?
        h_flat = tf.reshape(h, [batch*sequence, hparams.n_embd])
        logits = tf.matmul(h_flat, wte, transpose_b=True)
        logits = tf.reshape(logits, [batch, sequence, hparams.n_vocab])
        results['logits'] = logits
        return results